
**ACID**

- **Atomicity** - each statement in a transaction (to read, write, update or delete data) is treated as a single unit. Either the entire statement is executed, or none of it is executed. This property prevents data loss and corruption from occurring if, for example, if your streaming data source fails mid-stream.
- **Consistency** - ensures that transactions only make changes to tables in predefined, predictable ways. Transactional consistency ensures that corruption or errors in your data do not create unintended consequences for the integrity of your table.
- **Isolation** - when multiple users are reading and writing from the same table all at once, isolation of their transactions ensures that the concurrent transactions don't interfere with or affect one another. Each request can occur as though they were occurring one by one, even though they're actually occurring simultaneously.
- **Durability** - ensures that changes to your data made by successfully executed transactions will be saved, even in the event of system failure.

Cloud Storage started to replace the HDFS file for the Data Lake implementation. Its advantages include theoretically unlimited scale, pay-as-you-go billing, durability, and reliability. As a result, many organizations now use cloud object stores to manage large structured datasets in data warehouses and data lakes. 

Popular open-source systems, including Apache Spark or Presto, support reading and writing to cloud object stores using file formats such as Apache Parquet and ORC. 

Cloud services, including Google BigQuery and Redshift Spectrum, can also query directly against these systems and these open file formats.

Object storage has shortcomings when putting it into the Lakehouse context, which requires data warehouse capabilities right on top of the Datalake. 

Most cloud object storages are implemented as key-value stores, making achieving ACID transactions and high performance challenging: operations such as listing objects are expensive, and consistency guarantees are limited.

Assume we store relational data natively in the cloud object storage. We logically considered each table to be stored as a set of Parquet file objects; this approach creates correctness and performance challenges for more complex workloads:

- There is no isolation between queries: if a query needs to update multiple objects in the table, consumers will see partial updates as the query updates each object individually. here atomicity and consistency breaks(ACID)
- If an update query fails in the middle, the table is corrupted. durability fails here
- Metadata operations are expensive in large tables with millions of objects. (e.g., list)

ex: humare table m ek id h: 8097, hume iska status update krna h from open to closed

iske liye we have to scan all of the files of this table to find out this id, or atleast a bucket of ids ki files

haina, aur firr isme latest record find krna and update krna. 

now assume there are 2 ids, 8097 and 1023.

ho sakta h 8097 ekk file m ho aur 1023 kisi aur file m. 
1. file open
2. find data
3. if found update and close file
4. do we have more ids to update? if yes repeat step 1
5. if not data point found: close and open next file, go to step1 and repeat the process.




**Delta lake** :an ACID table storage layer on cloud object storage. The core idea of Delta Lake is straightforward:

> Maintain information about which objects belong to a Delta table in an ACID manner, using a write-ahead log in the cloud object store. The objects are encoded in Parquet, making it easy to write connectors from engines that can process Parquet.